### Creating NeRF-Compatiable training data (transforms.json)
#### Notebook Overview 

After doing feature mapping and feature extraction with COLMAP, we need to convert the extracted feature data into 
NeRF-Compatiable Format (aka transforms.json) to use in a training process.

STEPS :
1) Extract necessary data from associated COLMAP results - images.txt and cameras.txt
2) Calculate the Homography matrix and create a transforms.json file

#### Parse COLMAP Results and Create NeRF-Compatible Format

In [ ]:
from google.colab import files
import os
import cv2
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
# Function to parse COLMAP camera parameters
def parse_colmap_cameras(cameras_file):
    cameras = {}
    with open(cameras_file, "r") as f:
        for line in f:
            if line.startswith("#"):
                continue
            items = line.strip().split()
            if len(items) < 4:
                continue

            camera_id = int(items[0])
            model = items[1]
            width = int(items[2])
            height = int(items[3])
            params = np.array(list(map(float, items[4:])))

            cameras[camera_id] = {
                "id": camera_id,
                "model": model,
                "width": width,
                "height": height,
                "params": params
            }
    return cameras

# Function to parse COLMAP image parameters (camera poses)
def parse_colmap_images(images_file):
    images = {}
    with open(images_file, "r") as f:
        i = 0
        for line in f:
            if line.startswith("#"):
                continue
            if i == 0:  # Parse image data
                items = line.strip().split()
                image_id = int(items[0])
                qw, qx, qy, qz = map(float, items[1:5])
                tx, ty, tz = map(float, items[5:8])
                camera_id = int(items[8])
                image_name = items[9]

                images[image_id] = {
                    "id": image_id,
                    "qvec": [qw, qx, qy, qz],
                    "tvec": [tx, ty, tz],
                    "camera_id": camera_id,
                    "name": image_name,
                    "xys": [],
                    "point3D_ids": []
                }
                i = 1
            elif i == 1:  # Parse observed points
                i = 0
    return images

# Function to convert quaternion to rotation matrix
def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])

# Parse COLMAP output
try:
    camera_path = "./data/sparse/0/cameras.txt"
    images_path = "./data/sparse/0/images.txt"
    cameras = parse_colmap_cameras(camera_path)
    images = parse_colmap_images(images_path)

    print(f"Found {len(cameras)} cameras and {len(images)} images")

    # Print example camera parameters
    for camera_id, camera in list(cameras.items())[:1]:
        print(f"\nCamera {camera_id}:")
        print(f"  Model: {camera['model']}")
        print(f"  Width x Height: {camera['width']} x {camera['height']}")
        print(f"  Parameters: {camera['params']}")

    # Print example image parameters
    for image_id, image in list(images.items())[:1]:
        print(f"\nImage {image_id}: {image['name']}")
        print(f"  Camera ID: {image['camera_id']}")
        print(f"  Position: {image['tvec']}")
        print(f"  Rotation (quaternion): {image['qvec']}")

        # Convert to transformation matrix
        R = qvec2rotmat(image['qvec'])
        t = np.array(image['tvec'])

        # Create camera-to-world transformation matrix
        c2w = np.eye(4)
        c2w[:3, :3] = R
        c2w[:3, 3] = t

        print("  Transformation matrix (camera-to-world):")
        print(c2w)

except Exception as e:
    print(f"Error parsing COLMAP output: {e}")
    print("This might indicate that COLMAP failed to reconstruct the scene.")
    print("Try adjusting parameters or checking if the video contains enough distinct features.")

 #### Create NeRF-Compatible Transforms JSON File

In [ ]:
import json

def create_transforms_json(cameras, images, output_file="./data/transforms.json"):
    """Create a transforms.json file for NeRF training"""

    # Get the camera parameters from the first camera
    # Assuming all frames use the same camera
    camera_id = list(cameras.keys())[0]
    camera = cameras[camera_id]

    # For OPENCV camera model, the parameters are [fx, fy, cx, cy, k1, k2, p1, p2]
    # Extract focal length and principal point
    fx = camera['params'][0]
    fy = camera['params'][1]
    cx = camera['params'][2]
    cy = camera['params'][3]

    # Average focal length for simplicity
    # focal = (fx + fy) / 2

    frames = []
    for image_id, image in images.items():
        # Convert quaternion to rotation matrix
        R = qvec2rotmat(image['qvec'])
        t = np.array(image['tvec'])

        # Create camera-to-world transformation matrix
        c2w = np.eye(4)
        c2w[:3, :3] = R
        c2w[:3, 3] = t

        frame = {
            "file_path": "./frames/" + image['name'],
            "transform_matrix": c2w.tolist(),
        }
        frames.append(frame)

    # Create the full transforms dictionary
    transforms = {
        "camera_angle_x": 2 * np.arctan(camera['width'] / (2 * fx)),
        "camera_angle_y": 2 * np.arctan(camera['height'] / (2 * fy)),
        "fl_x": fx,
        "fl_y": fy,
        "cx": cx,
        "cy": cy,
        "w": camera['width'],
        "h": camera['height'],
        "frames": frames
    }

    # Save to file
    with open(output_file, 'w') as f:
        json.dump(transforms, f, indent=2)

    return transforms

# Create transforms.json
try:
    transforms = create_transforms_json(cameras, images)
    print("Created transforms.json")

    # Display the first few frames
    print("\nFirst frame transform:")
    print(json.dumps(transforms["frames"][0], indent=2))

    # Download the transforms.json file for later use
    files.download("transforms.json")

except Exception as e:
    print(f"Error creating transforms.json: {e}")